In [4]:
import pandas as pd
import os
import datetime
from datetime import time
import funciones as fun

In [5]:
# Lectura de las pestañas del excel, y limpieza de cada una de las tablas

nombres_hojas = ['Detalle_Factura','Detalle_Factura_1','HistoricoStock','HistoricoStock_1','Productos','Cabecera_Factura']

for nombre in nombres_hojas:
    contenido_limpio = fun.limpiar_hoja_excel(nombre)
    globals()[nombre] = contenido_limpio
    globals()[nombre].to_excel(nombre+'_limpio.xlsx', index=False)


KeyboardInterrupt



In [7]:
3 # Lectura y guardado de cada pestaña-

df_factura1 = pd.read_excel(r"C:\Users\innig\Documents\IDBOOTCAMP\TRABAJO FINAL\ARCHIVOS\Detalle_Factura_limpio.xlsx")
df_factura2 = pd.read_excel(r"C:\Users\innig\Documents\IDBOOTCAMP\TRABAJO FINAL\ARCHIVOS\Detalle_Factura_1_limpio.xlsx")

df_productos = pd.read_excel(r"C:\Users\innig\Documents\IDBOOTCAMP\TRABAJO FINAL\ARCHIVOS\Productos_limpio.xlsx")
df_cabecera = pd.read_excel(r"C:\Users\innig\Documents\IDBOOTCAMP\TRABAJO FINAL\ARCHIVOS\Cabecera_Factura_limpio.xlsx")

In [8]:
# Unimos factura 1 y 2, asi como stock 1 y 2.

facturas = pd.concat([df_factura1,df_factura2],axis=0)
productos = df_productos
cabecera = df_cabecera 

In [9]:
# Cambiamos el nombre de la columna del numero de factura para hacer los merges.

cabecera = cabecera.rename(columns = {"Numero":"Num_Factura"})

In [10]:
factura_total = pd.merge(facturas, cabecera[["Num_Factura","Fecha","Hora","Total"]], on="Num_Factura",how="left")

In [11]:
factura_final = factura_total.merge(productos[["Codigo","Coste","Seccion","MargenComercial"]],on="Codigo",how="left").sort_values("Num_Factura")

In [12]:
factura_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1250172 entries, 4644 to 1250038
Data columns (total 16 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Codigo           1250172 non-null  int64         
 1   Producto         1250172 non-null  object        
 2   Unidades         1250172 non-null  float64       
 3   Precio           1250172 non-null  float64       
 4   Num_Factura      1250172 non-null  int64         
 5   Precio_Unit      1250172 non-null  float64       
 6   CodMenu          1250172 non-null  object        
 7   codEmpleado      1250172 non-null  int64         
 8   NombreEmpleado   1250172 non-null  object        
 9   Tipo_Venta       1250172 non-null  object        
 10  Fecha            1250083 non-null  datetime64[ns]
 11  Hora             1250083 non-null  object        
 12  Total            1250083 non-null  float64       
 13  Coste            1243699 non-null  float64       
 14 

In [13]:
# Eliminamos las columnas innecesarias

In [14]:
factura_final.drop(["CodMenu","codEmpleado","NombreEmpleado","Tipo_Venta"],axis=1,inplace=True)

In [19]:
# Tratamos la columna hora y fecha para quedarnos con ellas limpias y poder juntarlas en una  de hora/fecha
# factura_final["Hora"] = factura_final["Hora"].str[0:8]
# factura_final["Fecha"] = factura_final["Fecha"].astype("str")
# factura_final["Fecha"] = factura_final["Fecha"].str[0:10]
## Aqui hacemos la funcion lambda para unir fecha y hora en una columna nueva, y cambiamos el tipo a datetime
# factura_final['Fecha_Hora'] = factura_final.apply(lambda x: ' '.join([str(x['Fecha']), str(x['Hora'])]), axis=1)
# factura_final["Fecha_Hora"] = factura_final['Fecha_Hora'].apply(lambda x: pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S'))

In [15]:
# Eliminamos los nan que tenemos en las columnas de hora, seccion, coste y margen

datos_fecha_nula = factura_final[factura_final.Hora.isnull()]
factura_final.drop(datos_fecha_nula.index,inplace=True)

In [16]:
datos_seccion_nulos = factura_final[factura_final.Seccion.isnull()]
factura_final.drop(datos_seccion_nulos.index,inplace=True)

In [18]:
factura_final

,Codigo,Producto,Unidades,Precio,Num_Factura,Precio_Unit,Fecha,Hora,Total,Coste,Seccion,MargenComercial
4644,1119,Fruta al peso,1.11,1.05,1,0.95,2015-08-16,22:43:00.019000,1.05,0.00,7.0,0.00
19432,1035,Cerveza 33cl SKOL,6.00,3.00,2,0.50,2015-08-16,22:59:00.010000,3.00,0.28,1.0,0.40
0,1031,"Cerveza sin 0,0 33cl SAN MIGUEL",1.00,0.70,3,0.70,2015-08-20,22:39:00,1.90,0.40,1.0,0.30
1,1060,"Agua 1,5L BEZOYA",1.00,0.70,3,0.70,2015-08-20,22:39:00,1.90,0.36,1.0,0.30
2,421,Tomate frito 265gr brick ORLANDO,1.00,0.50,3,0.50,2015-08-20,22:39:00,1.90,0.30,7.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...
1250034,7164,Donettes clasicos,1.00,1.99,467110,1.99,2022-11-01,21:00:00,7.84,1.27,7.0,0.23
1250037,1078,Chuches 0.05,2.00,0.10,467111,0.05,2022-11-01,21:29:59.971000,0.10,0.04,7.0,0.01
1250040,1223,Oferta 1€,1.00,1.00,467112,1.00,2022-11-01,21:40:00.019000,4.45,0.80,7.0,0.20
1250039,8689,Palmera bollo,1.00,1.25,467112,1.25,2022-11-01,21:40:00.019000,4.45,0.00,7.0,1.25


In [28]:
# Eliminamos la columna hora antiguas

In [19]:
factura_final.drop(["Hora"],axis=1,inplace=True)

In [20]:
factura_final

,Codigo,Producto,Unidades,Precio,Num_Factura,Precio_Unit,Fecha,Total,Coste,Seccion,MargenComercial
4644,1119,Fruta al peso,1.11,1.05,1,0.95,2015-08-16,1.05,0.00,7.0,0.00
19432,1035,Cerveza 33cl SKOL,6.00,3.00,2,0.50,2015-08-16,3.00,0.28,1.0,0.40
0,1031,"Cerveza sin 0,0 33cl SAN MIGUEL",1.00,0.70,3,0.70,2015-08-20,1.90,0.40,1.0,0.30
1,1060,"Agua 1,5L BEZOYA",1.00,0.70,3,0.70,2015-08-20,1.90,0.36,1.0,0.30
2,421,Tomate frito 265gr brick ORLANDO,1.00,0.50,3,0.50,2015-08-20,1.90,0.30,7.0,0.20
...,...,...,...,...,...,...,...,...,...,...,...
1250034,7164,Donettes clasicos,1.00,1.99,467110,1.99,2022-11-01,7.84,1.27,7.0,0.23
1250037,1078,Chuches 0.05,2.00,0.10,467111,0.05,2022-11-01,0.10,0.04,7.0,0.01
1250040,1223,Oferta 1€,1.00,1.00,467112,1.00,2022-11-01,4.45,0.80,7.0,0.20
1250039,8689,Palmera bollo,1.00,1.25,467112,1.25,2022-11-01,4.45,0.00,7.0,1.25


In [22]:
factura_final.to_csv('factura_final.csv',index=False)